In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import pytesseract
from mtcnn import MTCNN


# Load CSV
csv_path = "updated_data.csv"  # Replace with your updated CSV file path
df = pd.read_csv(csv_path)


# Limit to first 50 persons
# df_subset = df.iloc[234:255]
df_subset = df.head()

# Create output directory
output_dir = "extracted_id_faces"

# Initialize MTCNN detector
detector = MTCNN()

# Function for multi-angle detection
def detect_best_face(image):
    angles = [0, 90, 180, 270]
    best_result = None
    best_confidence = 0
    best_angle = 0
    best_img = image

    for angle in angles:
        if angle == 0:
            rotated_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        elif angle == 90:
            rotated_img = cv2.rotate(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), cv2.ROTATE_90_CLOCKWISE)
        elif angle == 180:
            rotated_img = cv2.rotate(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), cv2.ROTATE_180)
        else:  # 270
            rotated_img = cv2.rotate(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), cv2.ROTATE_90_COUNTERCLOCKWISE)

        faces = detector.detect_faces(rotated_img)
        if faces and faces[0]['confidence'] > best_confidence:
            best_confidence = faces[0]['confidence']
            best_result = faces[0]
            best_angle = angle
            best_img = rotated_img

    return best_result, best_angle, best_img, best_confidence  # Added best_confidence to return

# Process each ID image
for index, row in df_subset.iterrows():
    id_image_path = row['ID_IMAGE']
    person_id = row['ID']

    # Load image
    img = cv2.imread(id_image_path)
    if img is None:
        print(f"Error loading image: {id_image_path}")
        continue

    # Detect best face across all angles
    best_face, best_angle, img_to_annotate, best_confidence = detect_best_face(img)  # Updated to receive best_confidence
    print(f"Best detection for person {person_id} at {best_angle} degrees with confidence {best_confidence if best_face else 0}")

    # Annotate the image
    if best_face:
        x, y, w, h = best_face['box']
        x, y = max(0, x), max(0, y)

        # Draw bounding box (green)
        cv2.rectangle(img_to_annotate, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Draw landmarks (red dots)
        landmarks = best_face['keypoints']
        for key, point in landmarks.items():
            cv2.circle(img_to_annotate, point, 3, (0, 0, 255), -1)

        # Draw eye line (blue)
        left_eye = landmarks['left_eye']
        right_eye = landmarks['right_eye']
        cv2.line(img_to_annotate, left_eye, right_eye, (255, 0, 0), 2)
    else:
        # No face detected, draw a placeholder
        height, width = img_to_annotate.shape[:2]
        center = (width // 2, height // 2)
        cv2.rectangle(img_to_annotate, (center[0] - 50, center[1] - 50), (center[0] + 50, center[1] + 50), (0, 0, 255), 2, cv2.LINE_4)
        cv2.putText(img_to_annotate, "No Face Detected", (center[0] - 40, center[1] - 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Convert back to BGR for display and saving
    img_annotated = cv2.cvtColor(img_to_annotate, cv2.COLOR_RGB2BGR)

    # Display the annotated image
    # cv2.imshow(f"Person {person_id} - Press any key to continue", img_annotated)
    # print(f"Displaying person {person_id}. Check the bounding box, landmarks, and eye line.")

    # Save the annotated image
    output_path = os.path.join(output_dir, f"person_{person_id}_annotated.jpg")
    cv2.imwrite(output_path, img_annotated)
    print(f"Saved annotated image to {output_path}")

    # Wait for user input to continue
    print("Press any key to continue to the next image (or 'q' to quit)...")
    key = cv2.waitKey(0)
    if key & 0xFF == ord('q'):
        print("Quitting early.")
        break

# Close all windows
cv2.destroyAllWindows()
print("Visualization complete!")

Best detection for person 235 at 0 degrees with confidence 0.8145400285720825
Saved annotated image to extracted_id_faces\person_235_annotated.jpg
Press any key to continue to the next image (or 'q' to quit)...
Best detection for person 236 at 0 degrees with confidence 0.9989098310470581
Saved annotated image to extracted_id_faces\person_236_annotated.jpg
Press any key to continue to the next image (or 'q' to quit)...
Best detection for person 237 at 0 degrees with confidence 0.9982855916023254
Saved annotated image to extracted_id_faces\person_237_annotated.jpg
Press any key to continue to the next image (or 'q' to quit)...
Best detection for person 238 at 0 degrees with confidence 0
Saved annotated image to extracted_id_faces\person_238_annotated.jpg
Press any key to continue to the next image (or 'q' to quit)...
Best detection for person 239 at 0 degrees with confidence 0
Saved annotated image to extracted_id_faces\person_239_annotated.jpg
Press any key to continue to the next image

In [16]:
import os
import cv2
import pandas as pd
import numpy as np
from mtcnn import MTCNN

# Load CSV
csv_path = "updated_data.csv"  # Replace with your updated CSV file path
df = pd.read_csv(csv_path)

# Create output directories
output_dir = "extracted_id_faces"
log_dir = "extraction_logs"
for directory in [output_dir, log_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Initialize MTCNN detector
detector = MTCNN()
print("MTCNN initialized with default parameters for version 1.0.0.")

# Preprocessing function (less aggressive)
def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))  # Reduced clipLimit
    enhanced = clahe.apply(gray)
    denoised = cv2.GaussianBlur(enhanced, (3, 3), 0)  # Smaller kernel
    rgb_img = cv2.cvtColor(denoised, cv2.COLOR_GRAY2RGB)
    return rgb_img

# Function for multi-angle detection with improved validation
def detect_best_face(image):
    angles = [0, 180, 90, 270]
    best_result = None
    best_confidence = 0
    best_angle = 0
    best_img = image
    best_cropped_face = None

    for angle in angles:
        if angle == 0:
            rotated_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        elif angle == 90:
            rotated_img = cv2.rotate(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), cv2.ROTATE_90_CLOCKWISE)
        elif angle == 180:
            rotated_img = cv2.rotate(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), cv2.ROTATE_180)
        else:  # 270
            rotated_img = cv2.rotate(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), cv2.ROTATE_90_COUNTERCLOCKWISE)

        preprocessed_img = preprocess_image(rotated_img)
        faces = detector.detect_faces(
            preprocessed_img,
            # scale_factor=0.8,
            min_face_size=20,  # Adjusted to typical face size
            threshold_pnet=0.6,  # Stricter thresholds
            threshold_rnet=0.7,
            threshold_onet=0.8
        )
        if faces and faces[0]['confidence'] > best_confidence:
            best_confidence = faces[0]['confidence']
            best_result = faces[0]
            best_angle = angle
            best_img = rotated_img

            # Extract and validate bounding box
            x, y, w, h = best_result['box']
            x, y = max(0, x), max(0, y)
            # Ensure minimum size for cropping
            w, h = max(1, w), max(1, h)
            # Clip coordinates to image bounds
            x_end = min(best_img.shape[1], x + w)
            y_end = min(best_img.shape[0], y + h)
            x_start = max(0, x)
            y_start = max(0, y)

            # Crop the face
            if x_end > x_start and y_end > y_start:
                face_img = best_img[y_start:y_end, x_start:x_end]
                if face_img.size > 0:
                    best_cropped_face = cv2.resize(face_img, (224, 224))
                else:
                    with open(os.path.join(log_dir, "debug_log.txt"), 'a') as debug_log:
                        debug_log.write(f"Empty face_img for person {person_id} at angle {angle}: x={x_start}, y={y_start}, w={x_end-x_start}, h={y_end-y_start}, img_shape={best_img.shape}\n")
            else:
                with open(os.path.join(log_dir, "debug_log.txt"), 'a') as debug_log:
                    debug_log.write(f"Invalid crop dimensions for person {person_id} at angle {angle}: x_start={x_start}, y_start={y_start}, x_end={x_end}, y_end={y_end}, img_shape={best_img.shape}\n")

            if best_confidence > 0.9:
                break

    return best_result, best_angle, best_img, best_confidence, best_cropped_face

# Log file setup
log_file = os.path.join(log_dir, "extraction_log.txt")
with open(log_file, 'w') as log:
    log.write("Face Extraction Log\n")
    log.write("===================\n")

# Process all ID images
for index, row in df.iterrows():
    id_image_path = row['ID_IMAGE']
    person_id = row['ID']

    # Load image
    img = cv2.imread(id_image_path)
    if img is None:
        with open(log_file, 'a') as log:
            log.write(f"Error loading image for person {person_id}: {id_image_path}\n")
        continue

    # Detect best face across all angles
    best_face, best_angle, img_to_annotate, best_confidence, cropped_face = detect_best_face(img)
    with open(log_file, 'a') as log:
        log.write(f"Person {person_id}: Best detection at {best_angle} degrees with confidence {best_confidence if best_face else 0}\n")

    # Save the result
    if cropped_face is not None:
        output_path = os.path.join(output_dir, f"person_{person_id}_id_face.jpg")
        cv2.imwrite(output_path, cv2.cvtColor(cropped_face, cv2.COLOR_RGB2BGR))
        print(f"Extracted and saved face for person {person_id} at {output_path}")
    else:
        failed_path = os.path.join(output_dir, f"person_{person_id}_failed.jpg")
        cv2.imwrite(failed_path, img)
        print(f"No face detected for person {person_id}. Saved original image to {failed_path}")

print("Face extraction complete! Check extraction_log.txt and debug_log.txt for details.")

MTCNN initialized with default parameters for version 1.0.0.
Extracted and saved face for person 1 at extracted_id_faces\person_1_id_face.jpg
Extracted and saved face for person 2 at extracted_id_faces\person_2_id_face.jpg
Extracted and saved face for person 3 at extracted_id_faces\person_3_id_face.jpg
Extracted and saved face for person 4 at extracted_id_faces\person_4_id_face.jpg
Extracted and saved face for person 5 at extracted_id_faces\person_5_id_face.jpg
Extracted and saved face for person 6 at extracted_id_faces\person_6_id_face.jpg
Extracted and saved face for person 7 at extracted_id_faces\person_7_id_face.jpg
Extracted and saved face for person 8 at extracted_id_faces\person_8_id_face.jpg
Extracted and saved face for person 9 at extracted_id_faces\person_9_id_face.jpg
Extracted and saved face for person 10 at extracted_id_faces\person_10_id_face.jpg
Extracted and saved face for person 11 at extracted_id_faces\person_11_id_face.jpg
Extracted and saved face for person 12 at ex

In [19]:
import os
import pandas as pd

# Load the original CSV
csv_path = "updated_data.csv"  # Replace with your updated CSV file path
df = pd.read_csv(csv_path)

# Define directories
extracted_dir = "extracted_id_faces"
output_db_path = "face_id_database.csv"

# Create a dictionary mapping ID to Face_Photo path
id_to_face_photo = dict(zip(df['ID'], df['IMAGE']))

# Initialize list to store database entries
database_entries = []

# Scan extracted faces folder
for filename in os.listdir(extracted_dir):
    if filename.endswith("_id_face.jpg"):
        # Extract ID from filename (e.g., "person_123_id_face.jpg" -> "123")
        try:
            id_str = filename.replace("person_", "").replace("_id_face.jpg", "")
            person_id = int(id_str)
        except ValueError:
            print(f"Skipping invalid filename: {filename}")
            continue

        # Check if ID exists in the original CSV
        if person_id in id_to_face_photo:
            extracted_path = os.path.join(extracted_dir, filename)
            face_photo_path = id_to_face_photo[person_id]
            database_entries.append({
                "ID": person_id,
                "Extracted_Face_Path": extracted_path,
                "Face_Photo_Path": face_photo_path
            })
        else:
            print(f"ID {person_id} from {filename} not found in CSV, skipping.")

# Create DataFrame and save to CSV
if database_entries:
    db_df = pd.DataFrame(database_entries)
    db_df.to_csv(output_db_path, index=False)
    print(f"Database created successfully with {len(database_entries)} entries at {output_db_path}")
else:
    print("No valid extracted faces found to create database.")

print("Database creation complete!")

Database created successfully with 728 entries at face_id_database.csv
Database creation complete!


In [ ]:
# import os
# import cv2
# import pandas as pd
# import numpy as np
# from mtcnn import MTCNN

# # Load CSV
# csv_path = "updated_data.csv"  # Replace with your updated CSV file path
# df = pd.read_csv(csv_path)

# # Create output directory
# output_dir = "extracted_id_faces"
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# # Preprocessing and alignment function
# def preprocess_and_align_id_face(img):
#     # Convert to RGB for initial processing (handle colored ID image)
#     if len(img.shape) == 2:
#         img_rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
#     else:
#         img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#     # Detect face in the colored ID image
#     detector = MTCNN()
#     faces = detector.detect_faces(img_rgb)

#     if not faces:
#         print(f"No face detected initially, trying with adjusted parameters...")
#         detector.min_face_size = 20  # Reduce minimum face size
#         faces = detector.detect_faces(img_rgb)
#         if not faces:
#             print(f"Failed to detect face after adjustments.")
#             return None

#     # Use first detected face (black-and-white photo within ID)
#     x, y, w, h = faces[0]['box']
#     x, y = max(0, x), max(0, y)
#     face_img = img_rgb[y:y+h, x:x+w] if y+h <= img_rgb.shape[0] and x+w <= img_rgb.shape[1] else img_rgb[max(0, y):min(img_rgb.shape[0], y+h), max(0, x):min(img_rgb.shape[1], x+w)]

#     # Convert cropped face to grayscale (since it's black-and-white)
#     face_gray = cv2.cvtColor(face_img, cv2.COLOR_RGB2GRAY)

#     # Enhance lighting with CLAHE
#     clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
#     enhanced = clahe.apply(face_gray)
#     denoised = cv2.GaussianBlur(enhanced, (5, 5), 0)

#     # Get landmarks for alignment (eyes)
#     faces_gray = detector.detect_faces(cv2.cvtColor(denoised, cv2.COLOR_GRAY2RGB))
#     if not faces_gray or not faces_gray[0]['keypoints']:
#         print(f"No landmarks detected for alignment.")
#         return cv2.cvtColor(denoised, cv2.COLOR_GRAY2RGB)  # Fallback without rotation

#     landmarks = faces_gray[0]['keypoints']
#     left_eye = landmarks['left_eye']
#     right_eye = landmarks['right_eye']

#     # Calculate rotation angle based on eye positions
#     dy = right_eye[1] - left_eye[1]
#     dx = right_eye[0] - left_eye[0]
#     angle = np.degrees(np.arctan2(dy, dx)) - 180  # Align eyes horizontally

#     # Rotate image
#     center = (denoised.shape[1] // 2, denoised.shape[0] // 2)
#     rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
#     rotated_face = cv2.warpAffine(denoised, rotation_matrix, (denoised.shape[1], denoised.shape[0]))

#     # Convert back to RGB for output
#     rgb_face = cv2.cvtColor(rotated_face, cv2.COLOR_GRAY2RGB)
#     return cv2.resize(rgb_face, (224, 224))  # Resize to 224x224

# # Process each ID image in CSV
# for index, row in df.iterrows():
#     id_image_path = row['ID_IMAGE']
#     person_id = row['ID']

#     # Load image
#     img = cv2.imread(id_image_path)
#     if img is None:
#         print(f"Error loading image: {id_image_path}")
#         continue

#     # Extract and preprocess face
#     preprocessed_face = preprocess_and_align_id_face(img)
#     if preprocessed_face is not None:
#         # Save extracted face
#         output_path = os.path.join(output_dir, f"person_{person_id}_id_face.jpg")
#         cv2.imwrite(output_path, preprocessed_face)
#         print(f"Extracted and saved face for person {person_id} from {id_image_path}")
#     else:
#         print(f"Failed to process face for person {person_id} from {id_image_path}")

# print("Face extraction complete!")